In [7]:
import pandas as pd
dfOnline = pd.read_csv("../data/online_shoppers_intention.csv")
dfOnline.shape

(12330, 18)

Apply MinMaxEncoder or StandardScaler on the continuous features

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler1 = MinMaxScaler()
MinMax = pd.DataFrame(data = dfOnline[["Administrative_Duration", "Informational_Duration", "ProductRelated_Duration","BounceRates","ExitRates" ]])
dfMinMax = pd.DataFrame(data = scaler1.fit_transform(MinMax))
dfMinMax.columns = ["Administrative_Duration", "Informational_Duration", "ProductRelated_Duration","BounceRates","ExitRates"]

scaler2 = StandardScaler()
Standard = pd.DataFrame(data = dfOnline[["Administrative" ,"Informational","ProductRelated","PageValues"]])
dfStandard = pd.DataFrame(data = scaler2.fit_transform(Standard))
dfStandard.columns = ["Administrative" ,"Informational","ProductRelated","PageValues" ]

Apply OneHotEncoder or OrdinalEncoder on categorical features

In [9]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

dfOneHot = pd.DataFrame(dfOnline[["SpecialDay","Month","OperatingSystems","Browser","Region","TrafficType","VisitorType", "Weekend"]])
enc = OneHotEncoder(sparse=False,handle_unknown ='ignore' )
allOneHot = enc.fit_transform(dfOneHot)
colNames = list(enc.get_feature_names())
dfOneHot_new = pd.DataFrame(data = allOneHot)
dfOneHot_new.columns = colNames
#dfOneHot_new.head()
#dfOneHot
#print(dfOneHot.isnull().sum(axis=0)/dfOneHot.shape[0])

Apply the LabelEncoder on the target variable

In [10]:
from sklearn.preprocessing import LabelEncoder
tar =  LabelEncoder()
TaVar = list(dfOnline["Revenue"])
dfTaVar = pd.DataFrame(data = tar.fit_transform(TaVar))
dfTaVar.columns = ["Revenue"]


Combine all the preprocessed data together

In [ ]:
frames = [dfOneHot_new,dfMinMax,dfStandard,dfTaVar]
dfCombined = pd.concat(frames, axis = 1)
dfCombined.head()
dfCombined.to_csv('../data/prepocessedData.csv')

Check the missing values

In [16]:
print(dfCombined.isnull().sum(axis=0)/dfCombined.shape[0])

x0_0.0            0.000000
x0_0.2            0.000000
x0_0.4            0.000000
x0_0.6            0.000000
x0_0.8            0.000000
                    ...   
Administrative    0.001135
Informational     0.001135
ProductRelated    0.001135
PageValues        0.000000
Revenue           0.000000
Length: 81, dtype: float64
